In [ ]:
# --- 1. 환경 설정 ---

# 1. 필요한 라이브러리 설치
print("Installing necessary libraries...")
!pip install -q transformers ftfy regex tqdm scikit-learn

# 2. 모든 라이브러리 임포트
import os
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import CLIPModel, CLIPProcessor
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# 3. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ 1. 환경 설정 완료")

In [ ]:
# --- 2. 데이터셋 준비 ---

# 1. 데이터셋 경로 설정 및 압축 해제
DATASET_ZIP_PATH = "/content/drive/MyDrive/Datasets/EmoSet-118K.zip"
LOCAL_DATA_PATH = "/content/emoset"

if not os.path.exists(LOCAL_DATA_PATH):
    print(f"Unzipping dataset from {DATASET_ZIP_PATH} to {LOCAL_DATA_PATH}...")
    !unzip -q "{DATASET_ZIP_PATH}" -d "{LOCAL_DATA_PATH}"
    print("Unzipping complete.")
else:
    print(f"Dataset already exists at {LOCAL_DATA_PATH}.")


# 2. 이미지 전처리기(Transform) 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4815, 0.4578, 0.4082), (0.2686, 0.2613, 0.2758))
])
print("\nImage transform is ready.")


# 3. PyTorch Dataset 클래스 정의
class EmoSetDataset(Dataset):
    def __init__(self, json_path, base_path, transform=None):
        with open(json_path, "r") as f:
            raw_data = json.load(f)

        self.samples = [{'image_path': os.path.join(base_path, s[1]), 'label': s[0]} for s in raw_data]
        self.transform = transform

        self.labels = sorted(list({s['label'] for s in self.samples}))
        self.label2id = {label: i for i, label in enumerate(self.labels)}

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        img_path = sample['image_path']
        label_id = self.label2id[sample['label']]

        try:
            image = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            print(f"Warning: Image not found at {img_path}. Returning a black image.")
            image = Image.new("RGB", (224, 224)) # 파일이 없을 경우 대비

        if self.transform:
            image = self.transform(image)

        return image, label_id

print("PyTorch Dataset class 'EmoSetDataset' is defined.")


# 4. 데이터셋 및 데이터로더 인스턴스 생성
train_dataset = EmoSetDataset(json_path=os.path.join(LOCAL_DATA_PATH, "train.json"), base_path=LOCAL_DATA_PATH, transform=transform)
val_dataset = EmoSetDataset(json_path=os.path.join(LOCAL_DATA_PATH, "val.json"), base_path=LOCAL_DATA_PATH, transform=transform)

# DataLoader는 Dataset을 배치 단위로 묶어주고, 데이터를 섞어주는 역할을 함
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

# 이후 모델에서 사용할 클래스 정보 저장
NUM_CLASSES = len(train_dataset.labels)
CLASS_NAMES = train_dataset.labels

print(f"\nDataset and DataLoader are ready.")
print(f" - Found {NUM_CLASSES} classes: {CLASS_NAMES}")
print(f" - Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

print("\n✅ 2. 데이터셋 준비 완료")

In [ ]:
# --- 3. 모델 아키텍처 정의 ---

# 1. 커스텀 모델 클래스 정의
class CLIPForEmotionClassification(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # 1-1. Hugging Face Hub에서 사전 학습된 CLIP 모델("openai/clip-vit-base-patch32")로드
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

        # 1-2. CLIP 백본의 파라미터를 동결(freeze)
        for param in self.clip.parameters():
            param.requires_grad = False

        # 1-3. 분류기 헤드의 입력 차원을 CLIP 모델의 설정에서 자동으로 가져옴
        embedding_dim = self.clip.config.projection_dim

        # 1-4. 새로운 분류기 헤드 정의
        self.classifier = nn.Linear(embedding_dim, num_classes)

    def forward(self, pixel_values):
        # 2-1. 동결된 CLIP 모델로 이미지 특징 추출
        with torch.no_grad():
            image_embeds = self.clip.get_image_features(pixel_values=pixel_values)

        # 2-2. 추출된 특징을 학습 가능한 분류기 헤드에 통과시켜 최종 점수(logits) 획득
        logits = self.classifier(image_embeds)
        return logits

print("Custom model class 'CLIPForEmotionClassification' is defined.")
print("\n✅ 3. 모델 정의 완료")

In [ ]:
# --- 4. 훈련 준비 ---

# 1. 장치(Device) 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. 모델 인스턴스 생성
model = CLIPForEmotionClassification(num_classes=NUM_CLASSES).to(device)


# 3. 손실 함수(Loss Function) 정의
# 다중 클래스 분류 문제이므로, 표준적인 CrossEntropyLoss 사용
# 이 함수는 모델의 예측(logits)과 실제 정답(label) 사이의 차이를 측정하는 역할
criterion = nn.CrossEntropyLoss()


# 4. 옵티마이저(Optimizer) 정의
# model.parameters()가 아닌, model.classifier.parameters()를 전달하여 분류기 헤드의 가중치만 학습(업데이트)
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=1e-4) # 학습률(learning rate) = 1e-4


# 5. 평가 함수(Evaluation Function) 정의
@torch.inference_mode()
def evaluate(model, dataloader):
    model.eval() # 모델을 평가 모드로 설정 (Dropout 등을 비활성화)

    correct_predictions = 0
    total_samples = 0

    # dataloader로부터 배치 단위로 데이터 로드
    for images, labels in dataloader:
        # 데이터를 모델과 동일한 장치로 이동
        images, labels = images.to(device), labels.to(device)

        # 순전파를 통해 예측 수행
        outputs = model(images)

        # 가장 높은 점수(logit)를 가진 클래스를 예측 결과로 선택
        predicted_labels = outputs.argmax(dim=1)

        # 예측이 정답과 일치하는 개수 누적
        correct_predictions += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

    # 전체 정확도 계산 후 반환
    accuracy = correct_predictions / total_samples
    return accuracy

print("\nModel, Criterion, Optimizer, and Evaluation function are ready.")
print("\n✅ 4. 훈련 준비 완료")

In [ ]:
# --- 5. 모델 훈련 실행 ---

# 1. 훈련 하이퍼파라미터 설정
EPOCHS = 3
best_val_acc = 0.0
best_model_path = "/content/drive/MyDrive/image-music-recommendation-system_final/clip_emotion_classifier.pt" # 최고 성능 모델을 저장할 경로

# 훈련 과정 기록을 위한 딕셔너리
history = {'train_loss': [], 'train_acc': [], 'val_acc': []}

print(f"--- Starting training for {EPOCHS} epochs ---")

# 2. 훈련 및 검증 루프 실행
for epoch in range(EPOCHS):
    # --- 훈련 단계 (Training Phase) ---
    model.train() # 모델을 훈련 모드로 설정 (Dropout 등 활성화)

    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # tqdm을 사용하여 훈련 진행률 시각적 표시
    train_progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{EPOCHS} [Training]")

    for images, labels in train_progress_bar:
        # 데이터를 GPU로 이동
        images, labels = images.to(device), labels.to(device)

        # 옵티마이저의 기울기 초기화
        optimizer.zero_grad()

        # 순전파: 이미지를 모델에 입력하여 예측(logits) 생성
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 역전파: 손실을 기반으로 기울기 계산
        loss.backward()

        # 파라미터 업데이트: 계산된 기울기를 바탕으로 옵티마이저가 모델의 가중치를 수정
        optimizer.step()

        # 통계 기록
        running_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

        # 진행률 표시줄에 현재 배치 손실 표시
        train_progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})

    # --- 검증 단계 (Validation Phase) ---
    # Cell 4에서 정의한 evaluate 함수를 사용하여 검증 데이터셋에 대한 정확도 계산
    val_acc = evaluate(model, val_loader)

    # 한 에포크의 평균 손실과 훈련 정확도 계산
    train_loss = running_loss / len(train_loader)
    train_acc = correct_train / total_train

    # 훈련 과정 기록
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_acc'].append(val_acc)

    # --- 결과 출력 ---
    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
          f"Val Acc: {val_acc:.4f}")

    # --- 최고 성능 모델 저장 ---
    # 현재 검증 정확도가 기록된 최고 정확도보다 높으면, 모델의 가중치를 파일로 저장함
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"🎉 New best model saved with Val Acc: {best_val_acc:.4f} to {best_model_path}")

print(f"\n--- Training finished ---")
print(f"Best validation accuracy: {best_val_acc:.4f}")

print("\n✅ 5. 모델 훈련 완료")

In [ ]:
# --- 6. 훈련 결과 시각화 및 성능 분석 ---

# 1. 훈련 과정 그래프 시각화 (학습 곡선)
# 훈련 손실(Training Loss)과 검증 정확도(Validation Accuracy)의 변화를 에포크별로 시각화
print("--- Plotting Training and Validation Metrics ---")

if 'history' in locals():
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

    epochs = range(1, len(history['train_loss']) + 1)

    # 훈련 손실 그래프
    ax1.plot(epochs, history['train_loss'], marker='o', linestyle='-', color='b')
    ax1.set_title('Training Loss Over Epochs')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.grid(True)

    # 검증 정확도 그래프
    ax2.plot(epochs, history['val_acc'], marker='o', linestyle='-', color='r')
    ax2.set_title('Validation Accuracy Over Epochs')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.grid(True)

    plt.suptitle('Training and Validation Metrics', fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()
else:
    print("'history' dictionary not found. Please run the training cell (Cell 5) first.")


# 2. 혼동 행렬 (Confusion Matrix) 시각화
print("\n--- Generating Confusion Matrix for the fine-tuned model ---")

# 2-1. 예측 및 실제 레이블 수집을 위한 함수 정의
@torch.inference_mode()
def get_all_preds_and_labels(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    for images, labels in tqdm(dataloader, desc="Getting predictions for Confusion Matrix"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    return np.array(all_labels), np.array(all_preds)

# 2-2. 검증 데이터셋에 대한 예측 수행
# 최고 성능을 보인 모델의 가중치를 다시 불러와서 평가
print("Loading the best model for evaluation...")

# Cell 4에서 정의한 모델 구조를 그대로 사용
best_model = CLIPForEmotionClassification(num_classes=NUM_CLASSES).to(device)
best_model.load_state_dict(torch.load(best_model_path)) # Cell 5에서 저장한 최고 성능 모델 로드

true_labels, pred_labels = get_all_preds_and_labels(best_model, val_loader)

# 2-3. 혼동 행렬 계산 및 시각화
cm = confusion_matrix(true_labels, pred_labels)
# 각 클래스별 정답 개수로 나누어 정규화 (비율로 표시)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 10))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.title('Confusion Matrix (Normalized)', fontsize=16)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n✅ 6. 성능 시각화 및 분석 완료")

In [ ]:
# --- 7. Zero-Shot CLIP 모델과 성능 비교 ---
print("--- Evaluating the original Zero-Shot CLIP model for comparison ---")

# 1. Zero-Shot 예측을 위한 원본 CLIP 모델 로드
zeroshot_clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
zeroshot_clip_model.eval()

# 2. 텍스트 프롬프트 준비
# 제로샷 분류를 위해, 각 감정 클래스를 설명하는 텍스트 프롬프트 생성
text_prompts = [f"a photo that expresses {label}" for label in CLASS_NAMES]
text_inputs = processor(
    text=text_prompts,
    return_tensors="pt",
    padding=True
).to(device)

# 3. 제로샷 예측 수행 함수 정의
@torch.inference_mode()
def zeroshot_predict_labels(dataloader):
    all_preds = []
    all_labels = []

    text_features = zeroshot_clip_model.get_text_features(**text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    for images, labels in tqdm(dataloader, desc="Running Zero-Shot predictions"):
        images = images.to(device)

        # 이미지 특징 추출 및 정규화
        image_features = zeroshot_clip_model.get_image_features(pixel_values=images)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        # 코사인 유사도 계산
        logits = image_features @ text_features.T
        preds = logits.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    return np.array(all_labels), np.array(all_preds)

# 4. 검증 데이터셋에 대해 제로샷 예측 실행
true_zs, pred_zs = zeroshot_predict_labels(val_loader)

# 5. 혼동 행렬 비교 시각화
print("\n--- Comparing Confusion Matrices: Fine-tuned vs. Zero-shot ---")
if 'cm_normalized' not in locals():
    print("Running fine-tuned model prediction again for comparison...")
    true_labels, pred_labels = get_all_preds_and_labels(best_model, val_loader)
    cm = confusion_matrix(true_labels, pred_labels)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

cm_zs = confusion_matrix(true_zs, pred_zs)
cm_zs_norm = cm_zs.astype('float') / cm_zs.sum(axis=1)[:, np.newaxis]

fig, axes = plt.subplots(1, 2, figsize=(24, 10))
fig.suptitle('Confusion Matrix Comparison', fontsize=20)

# Fine-tuned 모델 혼동 행렬
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES, ax=axes[0])
axes[0].set_title('Fine-tuned Model', fontsize=16)
axes[0].set_xlabel('Predicted Label')
axes[0].set_ylabel('True Label')
plt.setp(axes[0].get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")


# Zero-shot 모델 혼동 행렬
sns.heatmap(cm_zs_norm, annot=True, fmt='.2f', cmap='Reds',
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES, ax=axes[1])
axes[1].set_title('Zero-shot CLIP (Original)', fontsize=16)
axes[1].set_xlabel('Predicted Label')
axes[1].set_ylabel('True Label')
plt.setp(axes[1].get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")


plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# 6. 최종 정확도 비교 출력
from sklearn.metrics import accuracy_score
print("\n--- Final Accuracy Comparison ---")
if 'true_labels' in locals() and 'pred_labels' in locals():
    print(f"Fine-tuned Model Accuracy: {accuracy_score(true_labels, pred_labels) * 100:.2f}%")
else:
    print("Fine-tuned results not found. Please run Cell 6 first.")
print(f"Zero-shot Model Accuracy:  {accuracy_score(true_zs, pred_zs) * 100:.2f}%")

print("\n✅ 7. 성능 비교 분석 완료")